In [71]:
#import astroprov

from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import wcs

from astroquery.simbad import Simbad

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
# from photutils import DAOStarFinder
# from photutils import CircularAperture
import os
import subprocess

import image_registration
from image_registration import chi2_shift
from image_registration.fft_tools import shift
import pandas
import collections

#from reproject import reproject_interp
from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS

from scipy.stats import iqr

In [8]:
imageDirectory = '/media/mj1e16/PP AV-TV/keplerCal/'
#medianDir = '/data/mj1e16/kepler/newDiff/'
dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0] != '.']
imagelist = [imageDirectory+x for x in dirlist]

In [4]:
def getSkyGroup(imagelist,image,channel):
    hdu_list = fits.open(imagelist[image])
    hdr = hdu_list[channel].header
    location = hdr['SKYGROUP']
    return location

In [32]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [6]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [9]:
extensionOrders = []
for image in imagelist:
    extensionOrders.append(correctExtensionOrder(image))

In [12]:
imagelist[1]

'/media/mj1e16/PP AV-TV/keplerCal/kplr2009114204835_ffi-cal.fits'

In [29]:
imageBunching = [[],[],[],[]]
extensions = []
for image in range(len(imagelist)):
    elist = extensionOrders[image].tolist()
    if elist not in extensions:
        extensions.append(elist)
        imageBunching[len(extensions)-1].append(imagelist[image])
    else:
        loc = extensions.index(elist)
        imageBunching[loc].append(imagelist[image])

In [30]:
imageBunching

[['/media/mj1e16/PP AV-TV/keplerCal/kplr2009114174833_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2009114204835_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2009115002613_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2009115053616_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2009115080620_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2009115131122_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2009115173611_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2009116035924_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2010111125026_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2010140101631_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2010174164113_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2011116104002_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2011145152723_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCal/kplr2011177110110_ffi-cal.fits',
  '/media/mj1e16/PP AV-TV/keplerCa

In [76]:
iqrFull = []
for imGroup in range(len(imageBunching)):
    imLoc = imagelist.index(imageBunching[imGroup][0])
    channels = extensionOrders[imLoc]
    iqrPerGroup = []
    for chan in range(1,len(channels)):
        imDataList = []
        for im in range(len(imageBunching[imGroup])):
            imDataList.append(getImageData(imageBunching[imGroup][im],int(channels[chan])))
        arrayStack = np.stack(imDataList)
        std = np.std(arrayStack,axis=0)
        interQRange = iqr(std)
        iqrPerGroup.append(interQRange)
    iqrFull.append(iqrPerGroup)
#     for im in range(len(imageBunching[imGroup])):
#         imLoc = 
#         for chan in range(len(extensionOrders[imLoc])):
            

In [77]:
iqrFull

[[1.2141180038452148,
  1.2303504943847656,
  1.860669732093811,
  1.2397860288619995,
  0.93486249446868896,
  1.0250508785247803,
  1.136622428894043,
  1.0248676538467407,
  0.85441696643829346,
  1.1170871257781982,
  1.3309148550033569,
  0.98126935958862305,
  1.423997163772583,
  1.605705738067627,
  1.7588746547698975,
  1.6927299499511719,
  1.2190742492675781,
  2.169905424118042,
  1.3048837184906006,
  1.3236408233642578,
  1.2552967071533203,
  1.3342082500457764,
  1.179779052734375,
  1.2161588668823242,
  1.011652946472168,
  1.06423020362854,
  1.1874926090240479,
  1.1730152368545532,
  1.0875879526138306,
  1.0920467376708984,
  1.2203153371810913,
  1.2048568725585938,
  1.9134922027587891,
  1.616774320602417,
  1.9521286487579346,
  2.1804609298706055,
  1.5862388610839844,
  1.5566596984863281,
  1.5092339515686035,
  2.0633835792541504,
  1.4783000946044922,
  1.2664475440979004,
  1.5749607086181641,
  1.4214355945587158,
  1.0824029445648193,
  1.2795250415802

In [68]:
std = np.std(arrayStack,axis=0)

In [72]:
interQRange = iqr(std)

In [73]:
interQRange

1.2141180038452148

In [59]:
a = np.zeros((2,2))
print(a)
b = np.zeros((2,2))
print(b)
c = np.dstack((a,b,a,b,a))

[[ 0.  0.]
 [ 0.  0.]]
[[ 0.  0.]
 [ 0.  0.]]


In [61]:
listofArrays = [a,a,a,a,a,a]
c= np.dstack(listofArrays)
c.shape

(2, 2, 6)

In [63]:
c = np.stack(listofArrays)

In [67]:
c.shape

(6, 2, 2)

In [60]:
d = np.append(c,b,axis=2)

ValueError: all the input arrays must have same number of dimensions

In [52]:
d = np.dstack((c,a))
print(d)

[[[ 0.  0.  0.]
  [ 0.  0.  0.]]

 [[ 0.  0.  0.]
  [ 0.  0.  0.]]]
